<a href="https://colab.research.google.com/github/pcsilcan/pcd/blob/master/20202/pcd_20202_0401_promelin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%script bash
git clone https://github.com/nimble-code/Spin.git
ls -l Spin
# :( esto no funcionó no ejecute esto por ahora.

total 36
drwxr-xr-x 2 root root 4096 Sep 11 12:45 Bin
drwxr-xr-x 2 root root 4096 Sep 11 12:45 Doc
drwxr-xr-x 5 root root 4096 Sep 11 12:45 Examples
-rw-r--r-- 1 root root 1669 Sep 11 12:45 LICENSE
-rw-r--r-- 1 root root   79 Sep 11 12:45 makefile
drwxr-xr-x 2 root root 4096 Sep 11 12:45 Man
drwxr-xr-x 2 root root 4096 Sep 11 12:45 optional_gui
-rw-r--r-- 1 root root 1522 Sep 11 12:45 README.md
drwxr-xr-x 2 root root 4096 Sep 11 12:45 Src


fatal: destination path 'Spin' already exists and is not an empty directory.


In [ ]:
!apt install spin

In [ ]:
%%script bash
#curl -o ps-programs.zip http://www.springer.com/cda/content/document/cda_downloaddocument/ps-programs.zip?SGWID=0-0-45-1050337-p173710973
unzip ps-programs.zip

In [21]:
%%writefile hola.pml
active[5] proctype P() {
    printf("%d Hola mundo\n", _pid)
}

Overwriting hola.pml


In [22]:
!spin hola.pml

                  3 Hola mundo
      0 Hola mundo
              2 Hola mundo
                      4 Hola mundo
          1 Hola mundo
5 processes created


In [36]:
%%writefile mtype.pml
mtype = { red, green, yellow }

mtype light = green

active proctype P() {
    byte i = 0;
    do
    :: i < 10 ->
        i++
        if
        :: light == red -> light = green
        :: light == green -> light = yellow
        :: light == yellow -> light = red
        fi
        printf("%e\n", light)
    :: else -> break
    od
}

Writing mtype.pml


In [37]:
!spin mtype.pml

      yellow
      red
      green
      yellow
      red
      green
      yellow
      red
      green
      yellow
1 process created


In [59]:
%%writefile if1.pml
active proctype P() {
    int a = 5, b = 5;
    int max;
    if
    :: a >= b -> max = a; printf("A es mayor\n")
    :: b >= a -> max = b; printf("B es mayor\n")
    fi
}

Overwriting if1.pml


In [58]:
!spin if1.pml

      B es mayor1 process created


In [62]:
%%writefile for.pml
active proctype P() {
    int i;
    for (i : 1..10) {
        printf("%d", i)
    }
    printf("\n")
}

Overwriting for.pml


In [63]:
!spin for.pml

      1      2      3      4      5      6      7      8      9      10      
1 process created


# Verificación

In [92]:
%%writefile counter.pml

byte n = 0

proctype P() {
    byte temp, i
    for (i : 1..10) {
        temp = n
        n = temp + 1
    }
}

init {
    atomic {
        run P();
        run P();
    }
    (_nr_pr == 1) -> printf("n = %d\n", n)
    assert(n > 2)
}

Overwriting counter.pml


In [96]:
!spin -run counter.pml

pan:1: assertion violated (n>2) (at depth 90)
pan: wrote counter.pml.trail

(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 28 byte, depth reached 92, errors: 1
    77329 states, stored
    28628 states, matched
   105957 transitions (= stored+matched)
        1 atomic steps
hash conflicts:        79 (resolved)

Stats on memory usage (in Megabytes):
    4.130	equivalent memory usage for states (stored*(State-vector + overhead))
    3.221	actual memory usage for states (compression: 77.99%)
         	state-vector as stored = 16 byte + 28 byte overhead
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  131.659	total actual memory usage



pan: elapsed time 0.03 seconds
pan: rate 2577633.3 states/second


In [98]:
%%script bash
spin -a counter.pml # crear un verificador en C
gcc pan.c           # los archivos pan son parte del verificador
./a.out             # ejecutando el verificador compilado

pan:1: assertion violated (n>2) (at depth 90)
pan: wrote counter.pml.trail

(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	acceptance   cycles 	- (not selected)
	invalid end states	+

State-vector 36 byte, depth reached 92, errors: 1
    77329 states, stored
    28628 states, matched
   105957 transitions (= stored+matched)
        1 atomic steps
hash conflicts:        46 (resolved)

Stats on memory usage (in Megabytes):
    4.720	equivalent memory usage for states (stored*(State-vector + overhead))
    3.221	actual memory usage for states (compression: 68.24%)
         	state-vector as stored = 16 byte + 28 byte overhead
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  131.659	total actual memory usage



pan: elapsed time 0.05 seconds
pan: rate   1546580 states/second


In [100]:
!spin -t counter.pml

      n = 2
spin: counter.pml:18, Error: assertion violated
spin: text of failed assertion: assert((n>2))
spin: trail ends after 91 steps
#processes: 1
		n = 2
 91:	proc  0 (:init::1) counter.pml:19 (state 7) <valid end state>
3 processes created
